## Env setup

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# For emptying trash after each run
"""from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')
drive_service.files().emptyTrash().execute()"""
!pwd
!nvidia-smi

## Imports

In [ ]:
folder_name = "/home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/" #"/content/drive/MyDrive/persuasion_technique_detection/"

In [ ]:
#!pip install transformers datasets wandb evaluate accelerate -qU sklearn_hierarchical_classification sentencepiece

In [ ]:
import gc
import re
import json
import numpy as np
import pandas as pd
import random
import torch
import subprocess
import json
import warnings
import shutil
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, ViTFeatureExtractor
from sklearn.metrics import f1_score, accuracy_score
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder
from sklearn.model_selection import train_test_split
from datasets import Dataset,load_dataset,DatasetDict,concatenate_datasets
import datasets
import os
from torch.nn.functional import sigmoid
from datasets import concatenate_datasets
from transformers import Trainer
from PIL import Image
import torch.nn.functional as F
from transformers import AutoModel, AutoImageProcessor,AutoTokenizer,AutoFeatureExtractor,ViTImageProcessor,ViTConfig, BertConfig, VisionTextDualEncoderConfig, VisionTextDualEncoderModel,CLIPImageProcessor

In [ ]:
import torch
AVAIL_GPUS = 0
if torch.cuda.is_available():
    device = torch.device("cuda")
    AVAIL_GPUS = torch.cuda.device_count()
    print(f'There are {AVAIL_GPUS} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

## Login WandB

In [ ]:
import wandb
import os

#wandb.login(relogin=True)
wandb.login()

# setup wandb environment variables
os.environ['WANDB_PROJECT'] = "subtask2b"
os.environ['WANDB_ENTITY'] = "tumnlp"
os.environ["WANDB_LOG_MODEL"]= "end"

In [ ]:
text_checkpoint = "vinai/bertweet-large"
img_checkpoint= "google/efficientnet-b5"
#"google/efficientnet-b5"
#"microsoft/resnet-152"
#"microsoft/resnet-50"
# "openai/clip-vit-base-patch32"
# openai/clip-vit-base-patch16
# google/vit-base-patch32-384
#google/vit-base-patch32-224-in21k
# google/vit-base-patch16-384
#'google/vit-base-patch16-224-in21k'

In [ ]:
summary_dir_path = folder_name + "subtask2b/CLS_summaries/summary_" + text_checkpoint.replace("/","_")+"_"+img_checkpoint.replace("/","_")+"/"

## Preprocess text input

In [ ]:
val_path="data/subtask2b/val.json"
train_path="data/subtask2b/train.json"
test_path="data/subtask2b/dev_unlabeled.json"
train_image_path="data/subtask2b/subtask2b_images/train/"
val_image_path="data/subtask2b/subtask2b_images/val/"
test_image_path="data/subtask2b/subtask2b_images/dev/"

with open(folder_name+val_path) as f:
  d = json.load(f)
  val=pd.DataFrame.from_dict(d)
  labels=val["label"]
  num_label=[int(el=="propagandistic") for el in labels]
  val["num_label"]=num_label
  val=val.drop(columns=['label'])
  val_set=val.rename(columns={"num_label": "label"})

with open(folder_name+train_path) as f:
  d = json.load(f)
  train=pd.DataFrame.from_dict(d)
  labels=train["label"]
  num_label=[int(el=="propagandistic") for el in labels]
  train["num_label"]=num_label
  train=train.drop(columns=['label'])
  train_set=train.rename(columns={"num_label": "label"})
  #mask = train_set['image'].isin(["prop_meme_24871.png"]) #also 'prop_meme_2758.png', 'prop_meme_2723.png' for texts removed
  #train_set = train_set[~mask]

with open(folder_name+test_path) as f:
  d = json.load(f)
  dev_unlabeled_set=pd.DataFrame.from_dict(d)

label2num={"non_propagandistic":0,"propagandistic":1}
num2label={0:"non_propagandistic",1:"propagandistic"}

print(len(train_set),len(val_set),len(dev_unlabeled_set))


In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, dataset_type,image_processor):
        super().__init__()
        self.ids=list(dataset["id"])
        self.texts = list(dataset["text"])
        self.image_paths = list(dataset["image"])
        if dataset_type=="train" or dataset_type=="val":
          self.labels = dataset["label"].astype(int).tolist()
        self.image_processor = image_processor
        self.dataset_type=dataset_type

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        if self.dataset_type=="train":
          image_path=folder_name+ train_image_path+ self.image_paths[idx]
        elif self.dataset_type=="val":
          image_path=folder_name+ val_image_path+ self.image_paths[idx]
        else:
          image_path=folder_name+ test_image_path+ self.image_paths[idx]

        image_input = self.image_processor(images=Image.open(image_path).convert("RGB"), return_tensors="pt")

        if self.dataset_type=="train" or self.dataset_type=="val":
          label = torch.tensor(self.labels[idx], dtype=torch.float32)
          return self.ids[idx],self.texts[idx],image_input,label
        else:
          return self.ids[idx],self.texts[idx],image_input

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(text_checkpoint)
text_model = AutoModel.from_pretrained(text_checkpoint)

# change image processor for different models
#image_processor = ViTImageProcessor.from_pretrained(img_checkpoint)  #'google/vit
#image_processor = CLIPImageProcessor.from_pretrained(img_checkpoint) #"openai/clip
image_processor = AutoImageProcessor.from_pretrained(img_checkpoint) # resnet, efficientnet
image_model = AutoModel.from_pretrained(img_checkpoint)
#image_model.config, text_model.config

In [ ]:
#image_model.config.hidden_dim,text_model.config.hidden_size

In [ ]:
class TextImageBinaryClassifier(nn.Module):
    def __init__(self, text_model, image_model):
        super(TextImageBinaryClassifier, self).__init__()
        self.text_model = text_model
        self.image_model = image_model

        #image_hidden_size=image_model.config.projection_dim #openai/vit
        #image_hidden_size=image_model.config.hidden_size # google/vit
        #image_hidden_size=image_model.config.hidden_sizes[-1] #resnet
        image_hidden_size=image_model.config.hidden_dim # efficientnet

        self.lin1 = nn.Linear(text_model.config.hidden_size + image_hidden_size,512)
        self.relu=nn.ReLU()
        self.lin2=nn.Linear(512,1)

    def forward(self, text_input, image_input):
        # Text encoding
        text_outputs = self.text_model(**text_input)

        text_embedding = text_outputs.last_hidden_state[:,0,:]

        # Image encoding
        try:
          #image_embedding=self.image_model.get_image_features(**image_input) #  "openai/clip

          #image_outputs = self.image_model(**image_input)  #'google/vit
          #image_embedding = image_outputs.last_hidden_state[:,0,:] #'google/vit

          image_outputs = self.image_model(**image_input) # resnet, efficientnet
          image_embedding=image_outputs.last_hidden_state # resnet, efficientnet
          image_embedding = F.adaptive_avg_pool2d(image_embedding, (1, 1)).view(image_embedding.size(0), image_embedding.size(1)) # resnet, efficientnet
          
        except Exception as e:
          print("error:",str(e))


        # Concatenate text and image embeddings
        combined_embedding = torch.cat((text_embedding, image_embedding), dim=1)


        x = self.lin1(combined_embedding)
        x=self.relu(x)
        logits=self.lin2(x)
        return logits

In [ ]:
# Create a custom dataset
train_dataset = CustomDataset(train_set,"train", image_processor)
val_dataset = CustomDataset(val_set,"val", image_processor)
test_dataset = CustomDataset(dev_unlabeled_set,"test", image_processor)

batch_size=2
num_workers=2
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,num_workers=num_workers,pin_memory=True)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=num_workers,pin_memory=True,drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,num_workers=num_workers,pin_memory=True)

In [ ]:
print("train size:",len(train_dataloader),"val size:",len(val_dataloader),"test size:",len(test_dataloader))

## Training

In [ ]:
transfer_learning = False
model_nodes = {
    "persuasion_or_not":None}
parent_nodes = {
    "persuasion_or_not":"persuasion_or_not"
}

In [ ]:
def train():
  try:
    # Initialize a new wandb run
    wandb.init()

    # sweep agent inputs config with hyperparameters
    config = wandb.config

    learning_rate = config.learning_rate
    run_name = config.run_name+f"_{str(learning_rate)}learningRate"
    wandb.run.name = run_name

    num_epochs = 10

    text_model = AutoModel.from_pretrained(text_checkpoint)

    image_model = AutoModel.from_pretrained(img_checkpoint)

    # Instantiate the custom model
    model = TextImageBinaryClassifier(text_model, image_model)
    model.cuda()
    loss_func = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    best_val_f1_macro=0.
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        all_preds = []
        all_labels = []
        print("Started training epoch:",epoch+1)
        for id, text_input, image_input, label in train_dataloader:
            optimizer.zero_grad()
            text_input = tokenizer(text_input, return_tensors="pt",truncation=True, padding=True,max_length=512)

            text_input=text_input.to("cuda")
            image_input['pixel_values']=image_input['pixel_values'].squeeze(1)
            image_input=image_input.to("cuda")
            label=label.to("cuda")
            logits = model(text_input, image_input)
            preds=sigmoid(logits)
            loss = loss_func(logits.squeeze(), label)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            # Collect predictions and labels for metrics calculation
            preds_classes = (preds > 0.5).int().cpu().numpy()
            all_preds.extend(preds_classes)
            all_labels.extend(label.cpu().numpy())

        average_loss = total_loss / len(train_dataloader)

        # Calculate and log F1 score and accuracy using sklearn metrics
        f1_macro = f1_score(all_labels, all_preds,average="macro")
        f1_micro = f1_score(all_labels, all_preds,average="micro")
        accuracy = accuracy_score(all_labels, all_preds)
        # Log metrics to W&B
        wandb.log({"epoch": epoch + 1, "train_loss": average_loss, "f1_macro": f1_macro, "f1_micro":f1_micro, "train_accuracy": accuracy})

        # Validation
        model.eval()
        val_loss = 0.0
        all_val_preds = []
        all_val_labels = []
        with torch.no_grad():
            for id, text_input, image_input, label in val_dataloader:
                text_input = tokenizer(text_input, return_tensors="pt",truncation=True, padding=True,max_length=512)

                text_input=text_input.to("cuda")
                image_input['pixel_values']=image_input['pixel_values'].squeeze(1)
                image_input=image_input.to("cuda")
                label=label.to("cuda")
                logits = model(text_input, image_input)
                preds=sigmoid(logits)
                val_loss += loss_func(logits.squeeze(), label).item()
                # Collect predictions and labels for metrics calculation
                val_preds_classes = (preds > 0.5).int().cpu().numpy()
                all_val_preds.extend(val_preds_classes)
                all_val_labels.extend(label.cpu().numpy())
        average_val_loss = val_loss / len(val_dataloader)

        # Calculate and log F1 score and accuracy for validation
        val_f1_macro = f1_score(all_val_labels, all_val_preds,average="macro")
        val_f1_micro = f1_score(all_val_labels, all_val_preds,average="micro")
        val_accuracy = accuracy_score(all_val_labels, all_val_preds)
        # Log metrics to W&B
        wandb.log({"epoch": epoch + 1, "val_loss": average_val_loss, "val_f1_macro": val_f1_macro,"val_f1_micro":val_f1_micro, "val_accuracy": val_accuracy})

        # Save the model if the current validation f1_macro is better than the previous best
        if val_f1_macro > best_val_f1_macro:
          best_val_f1_macro = val_f1_macro
          best_model_state_dict = model.state_dict()
          wandb.log({"eval/f1_macro":val_f1_macro})

        print(f"Epoch {epoch + 1}/{num_epochs}, Training f1-macro: {f1_macro}, Validation f1-macro: {val_f1_macro}")
        model.train()
    # Finish W&B run

    artifact = wandb.Artifact(f"best_model_{run_name}".replace("/","_"), type="model")
    artifact.add_file(folder_name+f"best_model_{run_name}.pth".replace("/","_"), torch.save(best_model_state_dict, folder_name+f"best_model_{run_name}.pth".replace("/","_")))
    wandb.run.log_artifact(artifact)
    os.remove(folder_name+f"best_model_{run_name}.pth".replace("/","_"))
    #drive_service.files().emptyTrash().execute()
    wandb.finish()
  except Exception as e:
    print(f"Error in training: {str(e)}")

In [ ]:
# Set hyperparams in sweep configurations
run_name=f'{text_checkpoint}-{img_checkpoint}-subtask2b-CLS'.replace("/","_")
sweep_name=f'sweep_{run_name}'
sweep_config = {
    'method': 'grid',  # can be grid, random, or bayes
    'name' : sweep_name,
    'metric': {
      'name': 'eval/f1_macro',
      'goal': 'maximize'
    },
    'parameters': {
        'learning_rate' : {
            'values': [1e-6,3e-6,5e-6,5e-5,5e-4]
        },
        'run_name': {
            'value' : run_name
        }
    }
}

# Start sweeps with specific configuration
sweep_id = wandb.sweep(sweep_config, project="subtask2b-CLS")
wandb.agent(sweep_id, train)
# Get best model of sweep
api = wandb.Api()
sweep = api.sweep(f"subtask2b-CLS/{sweep_id}")
best_run = sweep.best_run()

artifacts = best_run.logged_artifacts()

model_artifact = None
for artifact in artifacts:
    if 'model' in artifact.type:  # Adjust the condition based on your setup
        model_artifact = artifact
        break

if model_artifact != None:
  model_artifact_name = model_artifact.name
  print(f"Best Model: {model_artifact_name}")
else:
  warnings.warn(f"No models was found")

# save best model of this node
model_nodes["persuasion_or_not"] = model_artifact_name


In [ ]:
wandb.finish()

## Eval on val set and save run

In [ ]:
def write_json(path,data,file_name="summary.json"):
  if not isinstance(data, dict):
    data = data.to_dict("records")
  if not os.path.exists(path):
    os.makedirs(path)
  with open(path+file_name, "w") as output_file:
      json.dump(data, output_file, indent=2,ensure_ascii=False)

In [ ]:
summary = {
    "text_checkpoint" : text_checkpoint,
    "img_checkpoint":img_checkpoint,
    "model_nodes" : model_nodes,
    "train_path" : train_image_path,
    "val_path":val_image_path,
    "test_path":test_image_path
}

print(summary)
write_json(summary_dir_path,summary)

In [ ]:
write_json(summary_dir_path,summary)

### Eval on val set

In [ ]:
api = wandb.Api()
artifact=api.artifact(model_nodes["persuasion_or_not"])
model_dir=artifact.download()
model_state_dict_path = os.path.join(model_dir, model_nodes["persuasion_or_not"].split(":")[0]+".pth" )
model_state_dict = torch.load(model_state_dict_path)


text_model = AutoModel.from_pretrained(text_checkpoint)
image_model = AutoModel.from_pretrained(img_checkpoint)
# Instantiate the custom model
model = TextImageBinaryClassifier(text_model, image_model)
model.load_state_dict(model_state_dict)
model.cuda()

model.eval()

In [ ]:
loss_func = nn.BCEWithLogitsLoss()
val_loss = 0.0
all_val_preds = []
all_val_labels = []

with torch.no_grad():
    for id, text_input, image_input, label in val_dataloader:
        text_input = tokenizer(text_input, return_tensors="pt",truncation=True, padding=True,max_length=512)
        text_input=text_input.to("cuda")
        image_input['pixel_values']=image_input['pixel_values'].squeeze(1)
        image_input=image_input.to("cuda")
        label=label.to("cuda")
        logits = model(text_input, image_input)
        preds=sigmoid(logits)
        val_loss += loss_func(logits.squeeze(), label).item()

        # Collect predictions and labels for metrics calculation
        val_preds_classes = (preds > 0.5).int().cpu().numpy()
        all_val_preds.extend(val_preds_classes)
        all_val_labels.extend(label.cpu().numpy())

average_val_loss = val_loss / len(val_dataloader)

# Calculate and log F1 score and accuracy for validation
val_f1_macro = f1_score(all_val_labels, all_val_preds,average="macro")
val_f1_micro = f1_score(all_val_labels, all_val_preds,average="micro")
val_accuracy = accuracy_score(all_val_labels, all_val_preds)
val_results={"f1_macro":val_f1_macro,"f1_micro":val_f1_micro,"val_accuracy":val_accuracy}
print(val_results)
summary["val_results"]=val_results
print(summary)

val_pred_labels=[num2label[el.item()] for el in all_val_preds]
val_dataset_=val_set.drop(columns=["label","text","image"])
val_dataset_["label"]=val_pred_labels
write_json(summary_dir_path,val_dataset_,"val_preds.json")

In [ ]:
write_json(summary_dir_path, summary)

In [ ]:
def delete_dir(path):
    if os.path.exists(path):
        try:
            shutil.rmtree(path)
            print(f"The directory {path} and all its contents have been deleted successfully")
        except OSError as e:
            print(f"Error: {e.strerror}")
    else:
        print(f"The directory {path} does not exist")

In [ ]:
def delete_output_dirs(parent_directory):
    for entry in os.listdir(parent_directory):
        entry_path = os.path.join(parent_directory, entry)

        if os.path.isdir(entry_path) and entry.startswith('output_'):
            delete_dir(entry_path)

In [ ]:
"""path = folder_name + "subtask2b"
delete_output_dirs(path)
delete_dir(path + "tumnlp")
delete_dir(path + "wandb")
delete_dir(path + "artifacts")
delete_dir(path + "tmp_trainer")"""

## Predict on test set

In [ ]:
api = wandb.Api()
artifact=api.artifact(model_nodes["persuasion_or_not"])
model_dir=artifact.download()
model_state_dict_path = os.path.join(model_dir, model_nodes["persuasion_or_not"].split(":")[0]+".pth" )
model_state_dict = torch.load(model_state_dict_path)


text_model = AutoModel.from_pretrained(text_checkpoint)
image_model = AutoModel.from_pretrained(img_checkpoint)
# Instantiate the custom model
model = TextImageBinaryClassifier(text_model, image_model)
model.load_state_dict(model_state_dict)
model.cuda()

model.eval()

In [ ]:
all_preds = []

with torch.no_grad():
  for id, text_input, image_input in test_dataloader:
    text_input = tokenizer(text_input, return_tensors="pt",truncation=True, padding=True,max_length=512)
    text_input=text_input.to("cuda")
    image_input['pixel_values']=image_input['pixel_values'].squeeze(1)
    image_input=image_input.to("cuda")
    logits = model(text_input, image_input)
    preds=sigmoid(logits)

    # Collect predictions and labels for metrics calculation
    test_preds_classes = (preds > 0.5).int().cpu().numpy()
    all_preds.extend(test_preds_classes)

test_pred_labels=[num2label[el.item()] for el in all_preds]
dev_unlabeled_set["label"]=test_pred_labels
dev_unlabeled_set=dev_unlabeled_set.drop(columns=["text","image"])
write_json(summary_dir_path,dev_unlabeled_set,"dev_preds.json")

